In [339]:
%run ParserLexer.ipynb
import copy

In [340]:
def get_all_nodes(graph):
    sub_nodes = flatten([get_all_nodes(node.subfunc) for node in graph['nodes'] if getcn(node).startswith(("Reduce", "Map", "Iterate"))])
    return graph['nodes'] + sub_nodes
def get_all_channels(graph):
    relevant_nodes = [node for node in graph['nodes'] if getcn(node).startswith(("Reduce", "Map", "Iterate"))]
    sub_nodes = flatten([get_all_channels(node.subfunc) for node in relevant_nodes])
    for node in relevant_nodes:
        sub_nodes += [get_channel(node.output.name, x) for x in node.subfunc['inputs']]
    return graph['channels'] + sub_nodes


def magic_sv(i):
    myopts = [10,20,30,50,5,4]
    return random.choice(myopts)
    

In [341]:
def connected_ast(graph):
    nodes = copy.deepcopy(graph["nodes"])
    channels = copy.deepcopy(graph["channels"])
    def_len = 10
    
    # get  nodes
    masternodes = [n for n in nodes if getcn(n).startswith(("Reduce", "Map", "Iterate"))]
    # explode  nodes
    for node in masternodes:
        name = node.name
        da_in = node.input
        da_out = Port("out", name + "XDA_out", [1])
        da = Dearray(name + "XDA", da_in, da_out)
        ra_out = node.output
        ra_in = Port("in", name + "XRA_in", [1])
        ra = Rearray(name + "XRA", ra_in, ra_out, getcn(node))
        
        subgraph = connected_ast(node.subfunc)
        nodes += subgraph["nodes"]
        nodes += [da, ra]
        
        old_in_channel = [c for c in channels if c.dst_act == name]
        old_out_channel = [c for c in channels if c.src_act == name]
        assert(len(old_in_channel) <= 1)
        assert(len(old_out_channel) <= 1)
        if len(old_in_channel) == 1:
            old_in_channel[0].dst_act = da.name
        if len(old_out_channel) == 1:
            old_out_channel[0].src_act = ra.name
        
        channels += subgraph["channels"]
        so = subgraph["output"].name
        si = subgraph["inputs"][0].name
#         printls(subgraph['nodes'])
#         print(si
#         print(so)
        channels += [Channel(da.name, 
                             get_node_by_port(subgraph["nodes"], si).name, 
                             da_out.name, si), 
                     Channel(get_node_by_port(subgraph["nodes"], so).name, 
                             ra.name, so, ra_in.name)]
    # remove Map nodes
    nodes = [n for n in nodes if not getcn(n).startswith(("Reduce", "Map", "Iterate"))]
    in_ports = flatten([getin(n) for n in nodes])
    out_ports = flatten([getout(n) for n in nodes])
    my_inputs = [p for p in in_ports if p.name in [pp.name for pp in graph['inputs']]]
    my_output = [p for p in out_ports if p.name == graph['output'].name][0]
    
   
    
    return {
            "nodes" : nodes,
            "channels" : channels,
            "inputs" : my_inputs,
            "output" : my_output
        }


In [342]:
def get_all_nodes(graph):
    sub_nodes = flatten([get_all_nodes(node.subfunc) for node in graph['nodes'] if getcn(node).startswith(("Reduce", "Map", "Iterate"))])
    return graph['nodes'] + sub_nodes
def get_all_channels(graph):
    sub_channels = flatten([get_all_channels(node.subfunc) for node in graph['nodes'] if getcn(node).startswith(("Reduce", "Map", "Iterate"))])
    return graph['channels'] + sub_channels

In [343]:
def cascader(p, sv_maker = magic_sv):
    g = p['graph']
    ns = get_all_nodes(g)
    cs = get_all_channels(g)
    svs = dict([(p['sizevars'][i], sv_maker(i)) for i in range(len(p['sizevars']))])
    final_output = get_output_node(g).name
    print(final_output)
    cascade(p, ns, cs, svs, final_output )
    
def cascade(program, ns, cs, svs,final_output, triggers = {}):
    g = program["graph"]
    tmp_g = {'nodes':ns,'channels':cs}
    nd = dict([(n.name, n) for n in ns])
    cbsp = dict([(c.src_port, c) for c in cs])
    
    
    visited = []
    
    def form_type(s):
        if isinstance(s, TYPECLASS):
            return s
        op = getNextOp(s).lower()
        if op.startswith('float'):
            return Float()
        elif op.startswith('int'):
            return Int()
        elif op.startswith('array'):
            paras = getNextParams(s)
            assert(len(paras) == 2)
            sd = form_type(paras[0])
            try:
                l = int(paras[1].strip())
            except:
                l = svs[paras[1]] if 'SizeVar' not in paras[1] else magic_sv()
            return Array(l, sd)
        elif op.startswith('tuple'):
            paras = getNextParams(s)
            t = Tuple()
            [t.add_subdata(form_type(p)) for p in paras]
            return t
        else: 
            return Unknown()
    
    in_types = [(k, form_type(x) )for k, x in program["inputs"]]
    
    
    dearray_by = {}
    def def_odt(node, inc_dt):
        try:
            cn = getcn(node)
            if cn == "Param":
                if node.label == 'recursion':
                    node_num = re.findall(r'[0-9]+', node.name)[0]
                    rep_vec = dearray_by[node_num]
                    node.input.rate = [1] * rep_vec
                    node.output[0].rate = [1] * rep_vec
                    node.output[1].rate = para_out_1 = [0] * (rep_vec - 1) + [1]
                return inc_dt # e -> e
            elif cn.startswith(("Reduce", "Map")):
                print("SHOULDNT HAPPEN")
                cascaded_inputs = [(n.label, inc_dt.subdata) for n in get_input_nodes(node.subfunc)]
                bby_program = {'sizevars': program['sizevars'], 
                        'inputs': cascaded_inputs, 
                        'graph': node.subfunc}
                cascade(bby_program, ns, cs, svs)
                bby_outnode = get_output_node(bby_program['graph'])
                return bby_outnode.datatype
            elif cn == "Get":
                return inc_dt.subdatals[int(node.idx)]  #T(e, e, e, e...) -> e
            elif cn == "Transpose":
                sup_len = inc_dt.length
                sub_arr = inc_dt.subdata
                sub_len = sub_arr.length
                return Array(sub_len, Array(sup_len, sub_arr.subdata))
                # A( A(e, M), N ) -> A( A(e, N), M ) 
            elif cn == "Dearray":
                node_num = re.findall(r'[0-9]+', node.name)[0]
                node.output.rate = [inc_dt.length]
                dearray_by[node_num] = inc_dt.length
#                 print("DEARRAY ", node_num)
                return inc_dt.subdata # A(e, N) -> e
            elif cn == "Rearray":
                node_num = re.findall(r'[0-9]+', node.name)[0]
                new_len = 1 if node.masternode.startswith("Reduce") else dearray_by[node_num]
                node.input.rate = [new_len]
                return Array(new_len, inc_dt) # e -> A(e, ?)
            elif cn == "Join":
                sup_len = inc_dt.length
                sub_arr = inc_dt.subdata
                sub_len = sub_arr.length
                return Array(sub_len * sup_len,  sub_arr.subdata)
                # A( A(e, N), M) -> A(e, N*M)
            elif cn == "Value":
                return inc_dt
            elif cn == "Zip":
                lengths = [x.length for x in inc_dt]
                subdatas = [x.subdata for x in inc_dt]
#                 print(lengths)
                assert(len(set(lengths)) == 1)
                return Array(lengths[0], Tuple(subdatas))
                # A(e), A(r) -> A(T(e, r))
            elif cn == "Mather":
                return inc_dt # e, e -> e    
            elif cn == "UserFun":
                return Unknown() # e -> b
        except AttributeError as e:
            print("ERROR ERROR ERROR")
            if e.args[0] == "'Unknown' object has no attribute 'length'":
                return Unknown()
            else:
                raise
        except KeyError:
            default = 10
            if cn == "Param":
                node.input.rate = [1] * default
                node.output[0].rate = [1] * default
                node.output[1].rate = para_out_1 = [0] * (default - 1) + [1]
                return inc_dt # e -> e
            elif cn == "Rearray":
                node.input.rate = [default]
                return Array(default, inc_dt) # e -> A(e, ?)
            
    
                cascaded_inputs = [(n.label, inc_dt.subdata) for n in get_input_nodes(node.subfunc)]
                bby_program = {'sizevars': program['sizevars'], 
                        'inputs': cascaded_inputs, 
                        'graph': node.subfunc}
                cascade(bby_program, ns, cs, svs)
                bby_outnode = get_output_node(bby_program['graph'])
    
    
#     print(final_output)
    def push(node):
        print("Pushing: ", node.name)
        print("Triggers: ", [n.name for n in triggers.keys()])
        
        assert(node.has_dt())
        my_dt = node.get_dt()
        if node in triggers:
            print("helllllooooooo")
            masternode = triggers[node]
            length = masternode.rep if getcn(masternode) == 'Map' else 1
            masternode.add_dt(Array(length, my_dt))
            push(masternode)
            return
        if node.name in visited or node.name == final_output:
            return
        visited.append(node.name)
        print("Push ", node.name, getcn(node))
        
        output_nodes = [get_connected_node(tmp_g, out.name) for out in getout(node)]
        output_ports = getout(node)
        output_channels = [cbsp[p.name] for p in output_ports]
        output_nodes2 = [nd[c.dst_act] for c in output_channels]
        print(node.name,": to ",[n.name for n in output_nodes])
        assert(output_nodes == output_nodes2)
        
        for out_node in output_nodes:
            print(out_node.name)
            inc_nodes = [get_connected_node(tmp_g, inc.name) for inc in getin(out_node)]
            if not all([x.has_dt() for x in inc_nodes] ):
                print("          ", out_node.name, " not all")
                continue
            if getcn(out_node).startswith(("Map", "Reduce")):
                print("\nCASCADE")
                out_node.rep = my_dt.length
                cascaded_inputs = [(n.label, my_dt.subdata) for n in get_input_nodes(out_node.subfunc)]
                bby_program = {'sizevars': program['sizevars'], 
                        'inputs': cascaded_inputs, 
                        'graph': out_node.subfunc}
                bby_outnode = get_output_node(bby_program['graph'])
                triggers[bby_outnode] = out_node
                cascade(bby_program, ns, cs, svs,final_output, triggers)
                continue
            if getcn(out_node) != 'Zip':
                new_dt = def_odt(out_node, my_dt)
                print(out_node.name,", ",getcn(out_node)," : ",new_dt)
                out_node.add_dt(new_dt)
                push(out_node)
            else:
                my_dts = [get_connected_node(tmp_g, inc.name).datatype for inc in getin(out_node)]
                new_dt = def_odt(out_node, my_dts)
                print(out_node.name,", ",getcn(out_node)," : ",new_dt)
                out_node.add_dt(new_dt)
                push(out_node)
    
    
    # set types of input paramaters:
    starters = []
    for k, dt in in_types:
        param_nodes = [n for n in ns if getcn(n) == 'Param' and n.label == k]
        assert(len(param_nodes) == 1)
        param_node = param_nodes[0]
        print(param_node.name,", ",getcn(param_node)," : ",dt)
        param_node.add_dt(dt)
        starters.append(param_node)
    for starter in starters:
        push(starter)
    
    return program

# TODO: remove and fix actual cascade
def add_dt_to_channels(program):
    g = program['graph']
    cs = g["channels"]
    nd = dict([(n.name, n) for n in g["nodes"]])
    for c in cs:
        in_node = nd[c.src_act]
        in_dt = in_node.get_dt()
        c.add_dt(in_dt)
    return program

def add_datatypes(p, sizevar_file):
    sizevar_func = get_sizes_from_file(sizevar_file)
    cascade(p, sizevar_func)
    add_dt_to_channels(p)

In [344]:
p = test()
g = p['graph']
printgraph_simple(g)

n1: Param
n2: Param
n17: Map
[('n1', 'n17')]

       Subgraph of Map node  n17
n3: Param
n15: Map
n16: Transpose
[('n16', 'n15')]
[('n2', 'n16')]

       Subgraph of Map node  n15
n4: Param
n8: Reduce
n13: Map
n14: Zip
[('n13', 'n8')]
[('n3', 'n14')]
[('n4', 'n14')]
[('n14', 'n13')]

       Subgraph of Reduce node  n8
n5: Mather
n6: Param
n7: Param
[('n6', 'n5')]
[('n7', 'n5')]

       Subgraph of Map node  n13
n9: Param
n11: Get
n12: Get
n10: Mather
[('n9', 'n11')]
[('n11', 'n10')]
[('n9', 'n12')]
[('n12', 'n10')]


In [348]:
printgraph_simple(g, True)

n1: Param: Array (10, Array (50, Float))
n2: Param: Array (50, Array (50, Float))
n17: Map: Array (10, Array (50, Array (1, Float)))
[('n1', 'n17')]

       Subgraph of Map node  n17
n3: Param: Array (50, Float)
n15: Map: Array (50, Array (1, Float))
n16: Transpose: Array (50, Array (50, Float))
[('n16', 'n15')]
[('n2', 'n16')]

       Subgraph of Map node  n15
n4: Param: Array (50, Float)
n8: Reduce: Array (1, Float)
n13: Map: Array (50, Float)
n14: Zip: Array (50, Tuple (Float, Float))
[('n13', 'n8')]
[('n3', 'n14')]
[('n4', 'n14')]
[('n14', 'n13')]

       Subgraph of Reduce node  n8
n5: Mather: Float
n6: Param: Float
n7: Param: Float
[('n6', 'n5')]
[('n7', 'n5')]

       Subgraph of Map node  n13
n9: Param: Tuple (Float, Float)
n11: Get: Float
n12: Get: Float
n10: Mather: Float
[('n9', 'n11')]
[('n11', 'n10')]
[('n9', 'n12')]
[('n12', 'n10')]


In [347]:

cascader(p)

n17
n1 ,  Param  :  Array (10, Array (50, Float))
n2 ,  Param  :  Array (50, Array (50, Float))
Pushing:  n1
Triggers:  ['n15', 'n8', 'n10', 'n5']
Push  n1 Param
n1 : to  ['n17']
n17

CASCADE
n3 ,  Param  :  Array (50, Float)
Pushing:  n3
Triggers:  ['n15', 'n8', 'n10', 'n5']
Push  n3 Param
n3 : to  ['n14']
n14
n14 ,  Zip  :  Array (50, Tuple (Float, Float))
Pushing:  n14
Triggers:  ['n15', 'n8', 'n10', 'n5']
Push  n14 Zip
n14 : to  ['n13']
n13

CASCADE
n9 ,  Param  :  Tuple (Float, Float)
Pushing:  n9
Triggers:  ['n15', 'n8', 'n10', 'n5']
Push  n9 Param
n9 : to  ['n11', 'n12']
n11
n11 ,  Get  :  Float
Pushing:  n11
Triggers:  ['n15', 'n8', 'n10', 'n5']
Push  n11 Get
n11 : to  ['n10']
n10
n10 ,  Mather  :  Float
Pushing:  n10
Triggers:  ['n15', 'n8', 'n10', 'n5']
helllllooooooo
Pushing:  n13
Triggers:  ['n15', 'n8', 'n10', 'n5']
Push  n13 Map
n13 : to  ['n8']
n8

CASCADE
n6 ,  Param  :  Float
n7 ,  Param  :  Float
Pushing:  n6
Triggers:  ['n15', 'n8', 'n10', 'n5']
Push  n6 Param
n6 : t

In [292]:
s = g['nodes'][2].subfunc['nodes'][1].subfunc
s['nodes'][0].label

'bCol'

In [283]:
sf

{'nodes': [<__main__.Mather at 0x1125ebf10>],
 'channels': [],
 'inputs': [<__main__.Port at 0x1122c5950>, <__main__.Port at 0x1122c5910>],
 'output': <__main__.Port at 0x1122c5d50>}

In [155]:

ast = connected_ast(g)

In [156]:
astg = {'sizevars': copy.deepcopy(p['sizevars']), 
        'inputs': copy.deepcopy(p['inputs']), 
        'graph': ast}
add_datatypes(astg, None)

Push  n1 Param
n1
n15XDA
DEARRAY  15
Push  n15XDA Dearray
n15XDA
n3
Push  n3 Param
n3
n12
           n12  not all
Push  n2 Param
n2
n14
Push  n14 Transpose
n14
n13XDA
DEARRAY  13
Push  n13XDA Dearray
n13XDA
n4
Push  n4 Param
n4
n12
Push  n12 Zip
n12
n11XDA
DEARRAY  11
Push  n11XDA Dearray
n11XDA
n7
Push  n7 Param
n7
[<__main__.Get object at 0x111e50090>, <__main__.Get object at 0x111e50a10>]
[<__main__.Get object at 0x111e50090>, <__main__.Get object at 0x111e50a10>]
n9
Push  n9 Get
n9
n8
           n8  not all
n10
Push  n10 Get
n10
n8
Push  n8 Mather
n8
n11XRA
Push  n11XRA Rearray
n11XRA
n6XDA
DEARRAY  6
Push  n6XDA Dearray
n6XDA
n5


AttributeError: 'NoneType' object has no attribute 'has_dt'

In [106]:

ast = connected_ast(g)
printgraph_simple(g)

n1: Param
n2: Param
n15: Map
[('n1', 'n15')]

       Subgraph of Map node  n15
n3: Param
n13: Map
n14: Transpose
[('n14', 'n13')]
[('n2', 'n14')]

       Subgraph of Map node  n13
n4: Param
n6: Reduce
n11: Map
n12: Zip
[('n11', 'n6')]
[('n3', 'n12')]
[('n4', 'n12')]
[('n12', 'n11')]

       Subgraph of Reduce node  n6
n5: Mather

       Subgraph of Map node  n11
n7: Param
n9: Get
n10: Get
n8: Mather
[('n7', 'n9')]
[('n9', 'n8')]
[('n7', 'n10')]
[('n10', 'n8')]


In [107]:
printgraph_simple(ast)

n1: Param
n2: Param
n3: Param
n14: Transpose
n4: Param
n12: Zip
n5: Mather
n6XDA: Dearray
n6XRA: Rearray
n7: Param
n9: Get
n10: Get
n8: Mather
n11XDA: Dearray
n11XRA: Rearray
n13XDA: Dearray
n13XRA: Rearray
n15XDA: Dearray
n15XRA: Rearray
[('n1', 'n15XDA')]
[('n14', 'n13XDA')]
[('n2', 'n14')]
[('n11XRA', 'n6XDA')]
[('n3', 'n12')]
[('n4', 'n12')]
[('n12', 'n11XDA')]
[('n6XDA', 'n5')]
[('n5', 'n6XRA')]
[('n7', 'n9')]
[('n9', 'n8')]
[('n7', 'n10')]
[('n10', 'n8')]
[('n11XDA', 'n7')]
[('n8', 'n11XRA')]
[('n13XDA', 'n4')]
[('n6XRA', 'n13XRA')]
[('n15XDA', 'n3')]
[('n13XRA', 'n15XRA')]


In [70]:
printgraph(g)

<actor name='n1' type='Param' >
<port type='in' name='n1_in' rate='1'/>
<port type='out' name='n1_x0_out' rate='1'/>
</actor>
<actor name='n2' type='Param' >
<port type='in' name='n2_in' rate='1'/>
<port type='out' name='n2_x0_out' rate='1'/>
</actor>
<actor name='n15' type='Map'>
<port type='out' name='n15_out' rate='1'/>
<port type='in' name='n15_in' rate='1'/>
</actor>
<channel name='n1_x0_out_to_n15_in' srcActor='n1' srcPort='n1_x0_out' dstActor='n15' dstPort='n15_in' size='1' dtype='False' initialTokens='0'/>

       Subgraph of Map node  n15
<actor name='n3' type='Param' >
<port type='in' name='n3_in' rate='1'/>
<port type='out' name='n3_x0_out' rate='1'/>
</actor>
<actor name='n13' type='Map'>
<port type='out' name='n13_out' rate='1'/>
<port type='in' name='n13_in' rate='1'/>
</actor>
<actor name='n14' type='Transpose' >
<port type='in' name='n14_in' rate='1'/>
<port type='out' name='n14_out' rate='1'/>
</actor>
<channel name='n14_out_to_n13_in' srcActor='n14' srcPort='n14_out' 

In [99]:
g

{'nodes': [<__main__.Param at 0x11258f990>,
 'channels': [<__main__.Channel at 0x11256ca10>],
 'inputs': [<__main__.Port at 0x11258f710>, <__main__.Port at 0x11258fdd0>],
 'output': <__main__.Port at 0x11256cf10>}

In [103]:
print(ast['output'])

<port type='out' name='n15_out' rate='1'/>


In [102]:
printgraph(ast)

<actor name='n1' type='Param' >
<port type='in' name='n1_in' rate='1'/>
<port type='out' name='n1_x0_out' rate='1'/>
</actor>
<actor name='n2' type='Param' >
<port type='in' name='n2_in' rate='1'/>
<port type='out' name='n2_x0_out' rate='1'/>
</actor>
<actor name='n3' type='Param' >
<port type='in' name='n3_in' rate='1'/>
<port type='out' name='n3_x0_out' rate='1'/>
</actor>
<actor name='n14' type='Transpose' >
<port type='in' name='n14_in' rate='1'/>
<port type='out' name='n14_out' rate='1'/>
</actor>
<actor name='n4' type='Param' >
<port type='in' name='n4_in' rate='1'/>
<port type='out' name='n4_x0_out' rate='1'/>
</actor>
<actor name='n12' type='Zip'>
<port type='out' name='n12_out' rate='1'/>
<port type='in' name='n12_x0_in' rate='1'/>
<port type='in' name='n12_x1_in' rate='1'/>
</actor>
<actor name='n5' type='add'>
<port type='out' name='n5_out' rate='1'/>
<port type='in' name='n5_x0_in' rate='1'/>
<port type='in' name='n5_x1_in' rate='1'/>
</actor>
<actor name='n6XDA' type='Dear

In [21]:
printgraph(g)

<actor name='n1' type='Param' >
<port type='in' name='n1_in' rate='1'/>
<port type='out' name='n1_x0_out' rate='1'/>
</actor>
<actor name='n2' type='Param' >
<port type='in' name='n2_in' rate='1'/>
<port type='out' name='n2_x0_out' rate='1'/>
</actor>
<actor name='n15' type='Map'>
<port type='out' name='n15_out' rate='1'/>
<port type='in' name='n15_in' rate='1'/>
</actor>
<channel name='n1_x0_out_to_n15_in' srcActor='n1' srcPort='n1_x0_out' dstActor='n15' dstPort='n15_in' size='1' dtype='False' initialTokens='0'/>

       Subgraph of Map node  n15
<actor name='n3' type='Param' >
<port type='in' name='n3_in' rate='1'/>
<port type='out' name='n3_x0_out' rate='1'/>
</actor>
<actor name='n13' type='Map'>
<port type='out' name='n13_out' rate='1'/>
<port type='in' name='n13_in' rate='1'/>
</actor>
<actor name='n14' type='Transpose' >
<port type='in' name='n14_in' rate='1'/>
<port type='out' name='n14_out' rate='1'/>
</actor>
<channel name='n14_out_to_n13_in' srcActor='n14' srcPort='n14_out' 

In [54]:
g['nodes'][2].subfunc['nodes'][1].subfunc['nodes'][1].subfunc['inputs'][1].name

'n5_x1_in'

In [38]:
g['nodes'][2].output.name

'n15_out'

In [14]:
# Datatype management


def magic_sv(i):
    myopts = [10,20,30,50,5,4]
    return random.choice(myopts)

def get_sizes(ls):
    return lambda i: ls[i]

# get size vars
def get_sizes_from_file(fn = None):
    if fn:
        print("Hello")
        with open(fn) as f:
            ls = [int(x) for x in f.readlines()]
        return get_sizes(ls)
    return magic_sv
    
        

# Cascade data type and size information
  
# TODO: Refactor so that datatype is a function of channel, not node
# TODO: is there an okay way to rollback info gained after an Unknown?
# TODO: This is a godawful mess please fix
# I mean like duh
# NOTE! Must be done before smush_rede.

In [245]:
def tst(k):
    print(k)
    k['hi'] = 'world'
    if k['n'] % 6 == 0:
        k['m']  = k['m'] - 3
        k['n'] = k['n'] + 1
        tst(k)
    else:
        k['n'] = k['n'] + 1
    if k['n'] > 5:
        k['m'] = k['m'] + 2
    else:
        k['n'] = k['n'] + 1
        tst(k)
    if k['m'] < 10:
        tst(k)
    
tst({'m': 0,'n':0})

{'m': 0, 'n': 0}
{'m': -3, 'n': 1, 'hi': 'world'}
{'m': -3, 'n': 3, 'hi': 'world'}
{'m': -3, 'n': 5, 'hi': 'world'}
{'m': -1, 'n': 6, 'hi': 'world'}
{'m': -4, 'n': 7, 'hi': 'world'}
{'m': -2, 'n': 8, 'hi': 'world'}
{'m': 0, 'n': 9, 'hi': 'world'}
{'m': 2, 'n': 10, 'hi': 'world'}
{'m': 4, 'n': 11, 'hi': 'world'}
{'m': 6, 'n': 12, 'hi': 'world'}
{'m': 3, 'n': 13, 'hi': 'world'}
{'m': 5, 'n': 14, 'hi': 'world'}
{'m': 7, 'n': 15, 'hi': 'world'}
{'m': 9, 'n': 16, 'hi': 'world'}


In [248]:
d = {'a':'b'}
'b' in d
# 'a' not in d

False